# LAB10

Use reinforcement learning to devise a tic-tac-toe player.

### Deadlines:

* Submission: [Dies Natalis Solis Invicti](https://en.wikipedia.org/wiki/Sol_Invictus)
* Reviews: [Befana](https://en.wikipedia.org/wiki/Befana)

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

# !! WORK IN PROGRESS

In [36]:

import numpy as np

from itertools import combinations
from collections import defaultdict
from copy import copy
from tqdm.auto import tqdm
from random import random
from pprint import pprint

In [37]:
MAGIC = [2, 7, 6, 9, 5, 1, 4, 3, 8] # Magic square for 3x3 Tic-Tac-Toe

In [38]:


class TicTacToeState:
    '''
    Represents the state of a Tic-Tac-Toe game.
    
    Attributes:
    - x (list): The list of cells occupied by 'x'.
    - o (list): The list of cells occupied by 'o'.
    
    Methods:
    - get_board: Returns the current state of the board as a 3x3 numpy array.
    - get_actions: Returns the list of possible actions from the current state.
    - step: Returns the next state and reward after taking an action.
    - get_reward: Returns the reward for the current state.
    - is_terminal: Returns whether the current state is terminal.
    - is_win: Returns whether the current state is a win for the given player.
    - print_board: Prints the current state of the board.
    '''
    def __init__(self, x, o) -> None:
        self.x = x
        self.o = o
    
    def get_board(self):
        """
        Returns the current state of the board as a 3x3 numpy array.

        The board is represented by a list of length 9, where each element represents a cell on the board.
        The value 0 represents an empty cell, 1 represents a cell occupied by 'x', and 2 represents a cell occupied by 'o'.

        Returns:
        - np.ndarray: The current state of the board as a 3x3 numpy array.
        """
        board = [0 for _ in range(9)]
        for e in self.x:
            board[MAGIC.index(e)] = 1
        for e in self.o:
            board[MAGIC.index(e)] = 2
        
        return board

    def available_actions(self):
        """
        Returns the list of possible actions from the current state.

        The list of possible actions is represented by a list of length 9, where each element represents a cell on the board.
        The value 0 represents an empty cell, 1 represents a cell occupied by 'x', and 2 represents a cell occupied by 'o'.

        Returns:
        - list: The list of possible actions from the current state.
        """
        actions = []
        for i in range(9):
            if MAGIC[i] not in self.x and MAGIC[i] not in self.o:
                actions.append(i)
        return actions
    
    def not_available_actions(self):
        """
        Returns the list of impossible actions from the current state.

        The list of impossible actions is represented by a list of length 9, where each element represents a cell on the board.
        The value 0 represents an empty cell, 1 represents a cell occupied by 'x', and 2 represents a cell occupied by 'o'.

        Returns:
        - list: The list of impossible actions from the current state.
        """
        actions = []
        for i in range(9):
            if MAGIC[i] in self.x or MAGIC[i] in self.o:
                actions.append(i)
        return actions
    
    def step(self, action):
        """
        Returns the next state and reward after taking an action.

        Args:
        - action (int): The action to take.

        Returns:
        - (TicTacToeState, int): The next state and reward after taking an action.
        """
        if action not in self.available_actions():
            raise ValueError("Invalid action.")
        
        x = copy(self.x)
        o = copy(self.o)
        if len(x) == len(o):
            x.append(MAGIC[action])
        else:
            o.append(MAGIC[action])
            
        next_state = TicTacToeState(x, o)
        
        return next_state, next_state.get_reward()
    
    def is_win(self, player):
        """
        Returns whether the current state is a win for the given player.

        Args:
        - player (str): The player to check for a win.

        Returns:
        - bool: Whether the current state is a win for the given player.
        """
        if player == 'x':
            for c in combinations(self.x, 3):
                if sum(c) == 15:
                    return True
        else:
            for c in combinations(self.o, 3):
                if sum(c) == 15:
                    return True
        
        return False
    
    def get_reward(self):
        """
        Calculates the reward for the current state of the game.

        Returns:
        - float: The reward value.
        """
        if self.is_terminal():
            if len(self.x) > len(self.o):
                if self.is_win('x'):
                    return 100  # X wins
                else:
                    return 0  # Draw
            elif self.is_win('o'):
                return -100
            else:
                return 0
        else:
            reward = 0
            for c in combinations(self.x, 2):
                for a in set(MAGIC) ^ set(self.o):
                    if sum(c) + a == 15:
                        reward += 1

            for c in combinations(self.x, 1):
                for a in set(MAGIC) ^ set(self.o):
                    if sum(c) + a == 15:
                        reward += 0.5

            return reward
    
    def is_terminal(self):
        """
        Returns whether the current state is terminal.

        Returns:
        - bool: Whether the current state is terminal.
        """
        if len(self.x) + len(self.o) == 9:
            return True
        
        for c in combinations(self.x, 3):
            if sum(c) == 15:
                return True
        for c in combinations(self.o, 3):
            if sum(c) == 15:
                return True
        
        return False

    def print_board(self):
        """
        Prints the current state of the board.
        """
        board = np.array(self.get_board()).reshape(3, 3)
        for i in range(3):
            for j in range(3):
                if board[i][j] == 0:
                    print('.', end='')
                elif board[i][j] == 1:
                    print('X', end='')
                else:
                    print('O', end='')
            print()

In [39]:
class TicTacToeAgent:
    '''
    Represents a Tic-Tac-Toe agent.
    
    Attributes:
    - alpha (float): The learning rate.
    - gamma (float): The discount factor.
    - epsilon (float): The probability of taking a random action.
    - Q (defaultdict): The Q table that stores the action-value estimates.
    
    Methods:
    - take_action: Returns the action to take given the current state.
    - train: Updates the Q table given the current state, action, next state, and reward.
    '''
    def __init__(self, alpha=0.1, gamma=0.5, epsilon=0.1) -> None:
        '''
        Initializes a TicTacToeAgent object.

        Args:
        - alpha (float): The learning rate. Default is 0.1.
        - gamma (float): The discount factor. Default is 0.5.
        - epsilon (float): The probability of taking a random action. Default is 0.1.
        '''
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.Q = defaultdict(lambda: np.zeros(9))
           
    def take_action(self, state):
        """
        Returns the action to take given the current state.

        Args:
        - state (TicTacToeState): The current state.

        Returns:
        - int: The action to take given the current state.
        """
        self.Q[tuple(state.get_board())][state.not_available_actions()] = -100
        
        if random() < self.epsilon:
            return np.random.choice(state.available_actions())
        else:
            return np.argmax(self.Q[tuple(state.get_board())])
    
    def train(self, state, action, next_state, reward):
        """
        Updates the Q table given the current state, action, next state, and reward.

        Args:
        - state (TicTacToeState): The current state.
        - action (int): The action taken.
        - next_state (TicTacToeState): The next state.
        - reward (int): The reward received.
        """
        self.Q[tuple(state.get_board())][action] += self.alpha * (reward + self.gamma * np.max(self.Q[tuple(state.get_board())]) - self.Q[tuple(state.get_board())][action])

This code trains two TicTacToe agents using a reinforcement learning algorithm.
The first agent, `agent`, has an epsilon value of 0.1, which determines the exploration-exploitation trade-off.
The second agent, `agent_random`, has an epsilon value of 1, making it choose random actions.

The training loop runs for 150,000 iterations. In each iteration:
- A new TicTacToe state is created.
- The first agent takes an action based on its current state and updates its internal model.
- The next state and reward are obtained from the environment based on the chosen action.
- The first agent trains on the observed transition by updating its internal model.
- The state is updated to the next state.
- If the state is terminal, the loop breaks.
- The second agent takes an action based on the current state and updates its internal model.
- The next state and reward are obtained from the environment based on the chosen action.
- The first agent trains on the observed transition by updating its internal model.
- The state is updated to the next state.
- If the state is terminal, the loop breaks.

Note: The commented out lines that call `state.print_board()` can be used to print the TicTacToe board at each step.

In [40]:
agent = TicTacToeAgent(epsilon=0.1)
agent_random = TicTacToeAgent(epsilon=1)

for _ in tqdm(range(150_000)):
    state = TicTacToeState([], [])
    while not state.is_terminal():
        action = agent.take_action(state)
        next_state, reward = state.step(action)
        agent.train(state, action, next_state, reward)
        state = next_state
        #state.print_board()

        if state.is_terminal():
            break
        
        action = agent_random.take_action(state)
        next_state, reward = state.step(action)
        agent.train(state, action, next_state, -reward)
        state = next_state
        #state.print_board()

  0%|          | 0/150000 [00:00<?, ?it/s]

In [41]:
def calculate_percentage_of_x_wins(n_games):
    """
    Calculates the percentage of games won by 'x' player out of the total number of games played.
    
    Parameters:
    - n_games (int): The number of games to be played.
    
    Returns:
    - float: The percentage of games won by 'x' player.
    """
    
    assert n_games > 0
    
    x_wins = 0

    for _ in tqdm(range(n_games)):
        state = TicTacToeState([], [])
        while not state.is_terminal():
            action = agent.take_action(state)
            next_state, _ = state.step(action)
            state = next_state

            if state.is_terminal():
                break

            action = agent_random.take_action(state)
            next_state, _ = state.step(action)
            state = next_state

        if state.is_win("x"):
            x_wins += 1

    return x_wins / n_games * 100

In [42]:
percentage_of_x_wins = calculate_percentage_of_x_wins(100_000)
print("Percentage of games won by X: %.2f%%" % percentage_of_x_wins)


  0%|          | 0/100000 [00:00<?, ?it/s]

Percentage of games won by X: 90.84%
